In [1]:
import pandas as pd
import numpy as np
import requests
import io
from datetime import datetime

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


## Pruebas de carga local

## Pruebas de carga desde una URL

## Inicio

In [174]:
url_items = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat"
url_ratin = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat"
url_users = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/users.dat"

itm_file = pd.read_csv(url_items, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
use_file = pd.read_csv(url_users, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
rat_file = pd.read_csv(url_ratin, sep='::',encoding='utf8',error_bad_lines=False,engine='python')

itm_file.columns = ['movie_id', 'movie_title', 'genre']
use_file.columns = ['userid', 'twitter_id']
rat_file.columns = ['user_id','movie_id','rating','rating_timestamp']

In [175]:
itm_file.head() 

,movie_id,movie_title,genre
0,10,La sortie des usines Lumière (1895),Documentary|Short
1,12,The Arrival of a Train (1896),Documentary|Short
2,25,The Oxford and Cambridge University Boat Race ...,NaN
3,91,Le manoir du diable (1896),Short|Horror
4,131,Une nuit terrible (1896),Short|Comedy|Horror


In [176]:
itm_file.shape

(34398, 3)

In [177]:
use_file.head()

,userid,twitter_id
0,2,40501255
1,3,417333257
2,4,138805259
3,5,2452094989
4,6,391774225


In [178]:
use_file.shape

(60254, 2)

In [179]:
use_file = use_file.rename(columns = {'userid':'user_id'})

In [180]:
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
813776,60253,8695030,1,1567816456
813777,60253,9398640,1,1569026144
813778,60254,816711,8,1371972851
813779,60255,1559547,2,1373287369
813780,60255,2415464,2,1373772560


In [181]:
rat_file.shape

(813781, 4)

In [182]:
rat_file['rating_timestamp'] = pd.to_datetime(rat_file['rating_timestamp'], unit='s')
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
813776,60253,8695030,1,2019-09-07 00:34:16
813777,60253,9398640,1,2019-09-21 00:35:44
813778,60254,816711,8,2013-06-23 07:34:11
813779,60255,1559547,2,2013-07-08 12:42:49
813780,60255,2415464,2,2013-07-14 03:29:20


In [183]:
rat_file = rat_file.rename(columns = {'rating_timestamp':'date'})
rat_file['time'] = rat_file['date'].astype(str)
rat_file['date'] = rat_file['date'].astype(str)

In [184]:
rat_file.time.dtype

dtype('O')

In [185]:
rat_file['date'] = rat_file['date'].str.split(' ').str[0]
rat_file['time'] = rat_file['time'].str[-8:]

In [186]:
rat_file.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [187]:
itm_file['movie_year'] = itm_file['movie_title']

In [279]:
itm_file['movie_title'] = itm_file['movie_title'].str.split('(').str[0]
itm_file['movie_title'] = itm_file['movie_title'].str.strip()
itm_file['movie_year'] = itm_file['movie_year'].str[-6:]

In [280]:
itm_file['movie_year'] = itm_file['movie_year'].str.split('(').str[1]
itm_file['movie_year'] = itm_file['movie_year'].str.split(')').str[0]

In [281]:
itm_file.head() 

,movie_id,movie_title,genre,movie_year
0,10,La sortie des usines Lumière,Documentary|Short,1895
1,12,The Arrival of a Train,Documentary|Short,1896
2,25,The Oxford and Cambridge University Boat Race,NaN,1895
3,91,Le manoir du diable,Short|Horror,1896
4,131,Une nuit terrible,Short|Comedy|Horror,1896


In [282]:
itm_file.shape

(34398, 4)

In [283]:
df = rat_file

In [284]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [285]:
df['date']=df['date'].astype(str)

In [286]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [287]:
df['date']= pd.to_datetime(df['date']) 

In [288]:
df['date'].unique()

array(['2013-07-10T00:00:00.000000000', '2013-07-12T00:00:00.000000000',
       '2013-07-24T00:00:00.000000000', ...,
       '2014-12-11T00:00:00.000000000', '2015-09-25T00:00:00.000000000',
       '2015-09-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [289]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [290]:
df = df.sort_values(by='date')
df.reset_index(drop=True, inplace=True)
df.head()

,user_id,movie_id,rating,date,time
0,13922,825232,8,2013-02-28,18:02:44
1,50430,780653,6,2013-02-28,19:56:18
2,28067,302886,9,2013-02-28,23:21:31
3,33464,1795702,9,2013-02-28,22:56:53
4,54837,1428538,7,2013-02-28,17:11:18


In [291]:
df_2 = pd.merge(df,itm_file, on='movie_id')

In [292]:
df_2.head()

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year
0,13922,825232,8,2013-02-28,18:02:44,The Bucket List,Adventure|Comedy|Drama,2007
1,16059,825232,10,2013-03-03,07:37:55,The Bucket List,Adventure|Comedy|Drama,2007
2,46294,825232,8,2013-04-27,19:11:43,The Bucket List,Adventure|Comedy|Drama,2007
3,33508,825232,9,2013-05-01,12:12:58,The Bucket List,Adventure|Comedy|Drama,2007
4,36312,825232,6,2013-05-20,20:41:45,The Bucket List,Adventure|Comedy|Drama,2007


In [293]:
df_final =pd.merge(df_2, use_file, on='user_id')
df_final = df_final.sort_values(by='date')
df_final.reset_index(drop=True, inplace=True)
df_final

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id
0,13922,825232,8,2013-02-28,18:02:44,The Bucket List,Adventure|Comedy|Drama,2007,221804185
1,12013,1623288,8,2013-02-28,22:08:13,ParaNorman,Animation|Adventure|Comedy|Family|Fantasy,2012,47343120
2,29376,1013753,7,2013-02-28,22:20:30,Milk,Biography|Drama,2008,157612458
3,31268,1649419,7,2013-02-28,23:05:22,Lo imposible,Drama|History|Thriller,2012,374409594
4,30160,2125608,10,2013-02-28,20:56:31,Searching for Sugar Man,Documentary|Biography|Music,2012,36372507
5,37745,338013,4,2013-02-28,15:04:39,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi,2004,45826125
6,37745,118799,5,2013-02-28,15:00:53,La vita è bella,Comedy|Drama|Romance|War,1997,45826125
7,37745,64665,8,2013-02-28,15:07:27,Midnight Cowboy,Drama,1969,45826125
8,24603,2459022,7,2013-02-28,23:33:14,Lost in Thailand,Adventure|Comedy|Drama,2012,22073692
9,50430,780653,6,2013-02-28,19:56:18,The Wolfman,Drama|Fantasy|Horror|Thriller,2010,17804393


In [294]:
df_cross = pd.DataFrame(df_final.genre)

numeration = int(len(df_final))+1
df_cross['id'] = pd.Series(range(1,numeration))

In [295]:
df_cross

,genre,id
0,Adventure|Comedy|Drama,1
1,Animation|Adventure|Comedy|Family|Fantasy,2
2,Biography|Drama,3
3,Drama|History|Thriller,4
4,Documentary|Biography|Music,5
5,Drama|Romance|Sci-Fi,6
6,Comedy|Drama|Romance|War,7
7,Drama,8
8,Adventure|Comedy|Drama,9
9,Drama|Fantasy|Horror|Thriller,10


In [296]:
df_cross.isnull().values.any()

True

In [297]:
df_cross.isnull().sum()

genre    447
id         0
dtype: int64

In [298]:
df_cross['genre'] = df_cross.fillna('noGenre')

In [299]:
newdf_cross = pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in df_cross.iterrows()]).reset_index()

In [300]:
newdf_cross.columns

Index(['index', 0], dtype='object')

In [301]:
new_dfcross = pd.crosstab(newdf_cross[0], newdf_cross['index'])

In [302]:
new_dfcross

index,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [303]:
new_dfcross.rename(columns={'index': 'genre', 0: 'id'}, inplace=True)

In [304]:
new_dfcross.columns

Index(['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'noGenre'],
      dtype='object', name='index')

In [305]:
result = pd.merge(df_final.reset_index(),
                  new_dfcross.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

,index,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,0,13922,825232,8,2013-02-28,18:02:44,The Bucket List,Adventure|Comedy|Drama,2007,221804185,...,0,0,0,0,0,0,0,0,0,0
1,1,12013,1623288,8,2013-02-28,22:08:13,ParaNorman,Animation|Adventure|Comedy|Family|Fantasy,2012,47343120,...,0,0,0,0,0,0,0,0,0,0
2,2,29376,1013753,7,2013-02-28,22:20:30,Milk,Biography|Drama,2008,157612458,...,0,0,0,0,0,0,0,0,0,0
3,3,31268,1649419,7,2013-02-28,23:05:22,Lo imposible,Drama|History|Thriller,2012,374409594,...,0,0,0,0,0,0,1,0,0,0
4,4,30160,2125608,10,2013-02-28,20:56:31,Searching for Sugar Man,Documentary|Biography|Music,2012,36372507,...,0,0,0,0,0,0,0,0,0,0
5,5,37745,338013,4,2013-02-28,15:04:39,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi,2004,45826125,...,0,1,1,0,0,0,0,0,0,0
6,6,37745,118799,5,2013-02-28,15:00:53,La vita è bella,Comedy|Drama|Romance|War,1997,45826125,...,0,1,0,0,0,0,0,1,0,0
7,7,37745,64665,8,2013-02-28,15:07:27,Midnight Cowboy,Drama,1969,45826125,...,0,0,0,0,0,0,0,0,0,0
8,8,24603,2459022,7,2013-02-28,23:33:14,Lost in Thailand,Adventure|Comedy|Drama,2012,22073692,...,0,0,0,0,0,0,0,0,0,0
9,9,50430,780653,6,2013-02-28,19:56:18,The Wolfman,Drama|Fantasy|Horror|Thriller,2010,17804393,...,0,0,0,0,0,0,1,0,0,0


In [306]:
result.columns

Index([      'index',     'user_id',    'movie_id',      'rating',
              'date',        'time', 'movie_title',       'genre',
        'movie_year',  'twitter_id',             0,      'Action',
             'Adult',   'Adventure',   'Animation',   'Biography',
            'Comedy',       'Crime', 'Documentary',       'Drama',
            'Family',     'Fantasy',   'Film-Noir',   'Game-Show',
           'History',      'Horror',       'Music',     'Musical',
           'Mystery',        'News',  'Reality-TV',     'Romance',
            'Sci-Fi',       'Short',       'Sport',   'Talk-Show',
          'Thriller',         'War',     'Western',     'noGenre'],
      dtype='object')

#  Pruebas

In [44]:
df_testing = pd.DataFrame(df_final.genre)

pruebas = df_testing.head()
jj = int(pruebas.count()+1)
pruebas['id'] = pd.Series(range(1,jj))

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [45]:
pruebas

,genre,id
0,Adventure|Comedy|Drama,1
1,Animation|Adventure|Comedy|Family|Fantasy,2
2,Biography|Drama,3
3,Drama|History|Thriller,4
4,Documentary|Biography|Music,5


In [46]:
pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in pruebas.iterrows()]).reset_index()

,index,0
0,Adventure,1
1,Comedy,1
2,Drama,1
3,Animation,2
4,Adventure,2
5,Comedy,2
6,Family,2
7,Fantasy,2
8,Biography,3
9,Drama,3


In [47]:
import pandas as pd
df = pd.DataFrame(['a b c']*100000, columns=['col'])
df.head()

,col
0,a b c
1,a b c
2,a b c
3,a b c
4,a b c


In [48]:
print (pd.DataFrame(dict(zip(range(3), [df['col'].apply(lambda x : x.split(' ')[i]) for i in range(3)]))).head())

   0  1  2
0  a  b  c
1  a  b  c
2  a  b  c
3  a  b  c
4  a  b  c


In [49]:
s = pruebas['genre'].str.split('|')
s

0                         [Adventure, Comedy, Drama]
1    [Animation, Adventure, Comedy, Family, Fantasy]
2                                 [Biography, Drama]
3                         [Drama, History, Thriller]
4                    [Documentary, Biography, Music]
Name: genre, dtype: object

In [50]:
q = pruebas['genre'].str.split('|').tolist()
q

[['Adventure', 'Comedy', 'Drama'],
 ['Animation', 'Adventure', 'Comedy', 'Family', 'Fantasy'],
 ['Biography', 'Drama'],
 ['Drama', 'History', 'Thriller'],
 ['Documentary', 'Biography', 'Music']]

In [51]:
pd.DataFrame(dict(zip(range(16), [pruebas['genre'].apply(lambda x : x.split('|')[i]) for i in range(2)])))

,0,1
0,Adventure,Comedy
1,Animation,Adventure
2,Biography,Drama
3,Drama,History
4,Documentary,Biography


In [52]:
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s

0  0      Adventure
   1         Comedy
   2          Drama
1  0      Animation
   1      Adventure
   2         Comedy
   3         Family
   4        Fantasy
2  0      Biography
   1          Drama
3  0          Drama
   1        History
   2       Thriller
4  0    Documentary
   1      Biography
   2          Music
dtype: object

In [53]:
s.index = s.index.droplevel(-1)
s.name = 'genres'
s

0      Adventure
0         Comedy
0          Drama
1      Animation
1      Adventure
1         Comedy
1         Family
1        Fantasy
2      Biography
2          Drama
3          Drama
3        History
3       Thriller
4    Documentary
4      Biography
4          Music
Name: genres, dtype: object

In [54]:
# s = pruebas['genre']
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'genres'
pruebas.join(s.apply(lambda x: pd.Series(x.split('|'))))
pruebas = pruebas.join(s)
pruebas

,genre,id,genres
0,Adventure|Comedy|Drama,1,Adventure
0,Adventure|Comedy|Drama,1,Comedy
0,Adventure|Comedy|Drama,1,Drama
1,Animation|Adventure|Comedy|Family|Fantasy,2,Animation
1,Animation|Adventure|Comedy|Family|Fantasy,2,Adventure
1,Animation|Adventure|Comedy|Family|Fantasy,2,Comedy
1,Animation|Adventure|Comedy|Family|Fantasy,2,Family
1,Animation|Adventure|Comedy|Family|Fantasy,2,Fantasy
2,Biography|Drama,3,Biography
2,Biography|Drama,3,Drama


In [55]:
uni_gen = pruebas['genres'].unique()
uni_gen

array(['Adventure', 'Comedy', 'Drama', 'Animation', 'Family', 'Fantasy',
       'Biography', 'History', 'Thriller', 'Documentary', 'Music'],
      dtype=object)

In [56]:
process = pd.get_dummies(pd.Series(list(pruebas['genres'])))

In [57]:
result = pd.merge(pruebas.reset_index(),
                  process.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

,index_x,genre,id,genres,index_y,Adventure,Animation,Biography,Comedy,Documentary,Drama,Family,Fantasy,History,Music,Thriller
0,0,Adventure|Comedy|Drama,1,Adventure,0,1,0,0,0,0,0,0,0,0,0,0
1,0,Adventure|Comedy|Drama,1,Comedy,1,0,0,0,1,0,0,0,0,0,0,0
2,0,Adventure|Comedy|Drama,1,Drama,2,0,0,0,0,0,1,0,0,0,0,0
3,1,Animation|Adventure|Comedy|Family|Fantasy,2,Animation,3,0,1,0,0,0,0,0,0,0,0,0
4,1,Animation|Adventure|Comedy|Family|Fantasy,2,Adventure,4,1,0,0,0,0,0,0,0,0,0,0
5,1,Animation|Adventure|Comedy|Family|Fantasy,2,Comedy,5,0,0,0,1,0,0,0,0,0,0,0
6,1,Animation|Adventure|Comedy|Family|Fantasy,2,Family,6,0,0,0,0,0,0,1,0,0,0,0
7,1,Animation|Adventure|Comedy|Family|Fantasy,2,Fantasy,7,0,0,0,0,0,0,0,1,0,0,0
8,2,Biography|Drama,3,Biography,8,0,0,1,0,0,0,0,0,0,0,0
9,2,Biography|Drama,3,Drama,9,0,0,0,0,0,1,0,0,0,0,0


In [58]:
new_df = pd.crosstab(result['id'], result['genres'])

In [59]:
new_df

genres,Adventure,Animation,Biography,Comedy,Documentary,Drama,Family,Fantasy,History,Music,Thriller
id,,,,,,,,,,,
1,1,0,0,1,0,1,0,0,0,0,0
2,1,1,0,1,0,0,1,1,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,1
5,0,0,1,0,1,0,0,0,0,1,0


In [60]:
data = pd.DataFrame(columns=(uni_gen))
data['id'] = pd.Series(range(1,jj))
data

,Adventure,Comedy,Drama,Animation,Family,Fantasy,Biography,History,Thriller,Documentary,Music,id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [62]:
df_final.shape

(813868, 9)

In [63]:
df_final['movie_title'].nunique()
#34314

33021

In [307]:
lista_m = itm_file['movie_title']

lista_f = df_final['movie_title']

In [ ]:
df_final[['dID','hID']].groupby(df['mID']).agg(['count', 'size', 'nunique'])

In [ ]:
lista_m.iloc['Joker']

is_male = lista_m.loc[:,] == 'Joker'
df_male = lista_m.loc[is_male]
df_male.head()

In [309]:
lista_f = pd.DataFrame(lista_f)
lista_f

,movie_title
0,The Bucket List
1,ParaNorman
2,Milk
3,Lo imposible
4,Searching for Sugar Man
5,Eternal Sunshine of the Spotless Mind
6,La vita è bella
7,Midnight Cowboy
8,Lost in Thailand
9,The Wolfman


In [310]:
#surveys_df.loc[[0, 10], :] surveys_df[surveys_df.year == 2002]

lista_m = pd.DataFrame(lista_m)
lista_m['movie_title'].iloc[2]

'The Oxford and Cambridge University Boat Race'

In [311]:
lista_m.movie_title.dtypes

dtype('O')

In [313]:
lista_m.movie_title =='The Oxford and Cambridge University Boat Race'

0        False
1        False
2         True
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
34368    False
34369    False
34370    False
34371    False
34372    False
34373    False
34374    False
34375    False
34376    False
34377    False
34378    False
34379    False
34380    False
34381    False
34382    False
34383    False
34384    False
34385    False
34386    False
34387    False
34388    False
34389    False
34390    False
34391    False
34392    False
34393    False
34394    False
34395    False
34396    False
34397    False
Name: movie_title, Length: 34398, dtype: bool

In [314]:
is_joker = lista_m.loc[:, 'movie_title'] == 'Upstarts'
df_joker = lista_m.loc[is_joker]
df_joker.head()

,movie_title
34397,Upstarts


In [315]:
is_joker.unique()

array([False,  True])

In [316]:
comparacion = []
for item in lista_m:
    if item in lista_m:
        comparacion.append(item)

if len(comparacion) > 0:
    print('Ambas listas contienen estos elementos')
    for item in comparacion: print('%s' % item)

else:
    print('No existe ningun elemento igual en las listas')

Ambas listas contienen estos elementos
movie_title


In [ ]:
lista1=["paco","pepe","luis"]

lista2=["diego","mari","luis"]

comparacion = []

 

for item in lista1:

  if item in lista2:

    comparacion.append(item)

 

if len(comparacion) > 0:

  print 'Ambas listas contienen estos elementos'

  for item in comparacion: print '%s' % item

else:

  print 'No existe ningun elemento igual en las listas'

In [ ]:
n_col = df_final.columns

for col in n_col:
    print(df[col].unique)